# Compte rendu TP n°2 - TC6
                                                                                                        Des Lauriers Cédric
                                                                                                        Cornille Théo

**Dataset: Billion words imputation**

Seulement une partie du corpus a été utilisé pour le jeu d'entraînement correspondant aux 2000 premières lignes du dataset initial.
Le modèle ne prédit que les mots qui appartiennent au jeu d'entraînement.
Nous allons donc évaluer le modèle bi-gramme dans ce compte rendu


In [13]:
# On a utilisé seulement une petite parti du corpus pour le train , seulement les 2000 premières lignes, car les temps de calcul
# était trop long ,  donc le modèle peut ne pas prédire de mot notament si les mots de phrases test ne sont pas compris dans le
# le vocabulaire du train , au delà de ça le modèle bigram marche bien 

Importation des bibliothèques

In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.sql.functions import desc
from pyspark.sql.types import *
from pyspark.sql.types import *
from pyspark.sql import SQLContext



In [2]:
import pyspark
from pyspark import SparkContext

#SparkContext.setSystemProperty('spark.executor.memory', '8g')
sc =SparkContext(appName="Missing word")
sqlContext = SQLContext(sc)


In [3]:
sc._conf.getAll()

[('spark.driver.port', '59100'),
 ('spark.driver.host', '192.168.1.15'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'local-1543146266346'),
 ('spark.app.name', 'Missing word'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

# Conception des bases de données : train
On crée ici les matrices de transition contenant les probabilités (ici sous formes de dataframe spark) de nos notre modèle.
 
### Pour le modèle bigram : P(mot_i+1|mot_i), et P(tag_i+1|tag_i)

#### On nettoie le texte , on quantifie les nombres / scores , charactères spéciaux , dates , numéro de tel ,etc .

In [7]:
import re

# si par exemple le mot manquant est au début ou à la fin : mot1 mot2 => debut [missing_word] mot1 mot2 
# la fonction balise permet d'utiliser aisement le modele bi-gramme  argmax P(mot_test|debut)* P(mot1|mot_test)
# ou encore déterminer l'indice : argmin P(tag1|debut), on verra cela en détail dans la suite 
def balise(text):
    s="DEBUT"
    s= s+ " "+text+ " "+ "FIN"
    return s

def parseAndRemoveStopWords(text):
    text = re.sub(r'http\S+', 'url', text, flags=re.MULTILINE) # supimme url 
    text = re.sub(r"\w{3}-\w{3}-\w{4}", "phonenumber", text, flags=re.MULTILINE)
    
    text = re.sub(r"([12]\d{3}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01]))", "datenumber", text, flags=re.MULTILINE)
    
    text = re.sub(r"\b(0|[1-9]\d*)-(0|[1-9]\d*)\b", "score", text, flags=re.MULTILINE)
    text = re.sub(r"[0-9]{5,}", "bignumber", text, flags=re.MULTILINE)
    text = re.sub(r"[0-9]{1,}th|[0-9]{1,}st", "numberrank", text, flags=re.MULTILINE)
    
    text = re.sub(r"[0-9]{4}", "numberfour", text, flags=re.MULTILINE)
    text = re.sub(r"[0-9]{1,3}bn", "littlenumber bn", text, flags=re.MULTILINE)
    text = re.sub(r"[0-9]{1,3}", "littlenumber", text, flags=re.MULTILINE)

    t = text.replace(";","").replace(":","").replace(' "','').replace('"','').replace('-','').replace(' ?','').replace("?","")
    t = t.replace(',','').replace(' .','').replace('.','').replace(' !','').replace('!','').replace(" '","").replace("'","").replace("/","").replace("\\","").replace("","")
    t = t.replace(')','').replace('(','').replace('  ',' ')  #t.lower()
    t = re.sub(r"[^\x00-\x7F]", "", t) # Non ASCII charactères
    t = re.sub(r"\s+", " ", t)
    if t[0]==" ":
        t.replace(t[0] ,"")
    if t[-1] == " ":
        t.replace(t[-1] ,"")

    return t


In [4]:
essai_string = "sazoezd,fpz, 98th vdvf ( fzf )  68bn 81st d,e fdze,nkfn 111111111111 1993-08-19  1 lol 12 dedez 123 kednzln 123-456-7890 7-8  26-28 76 1-2  2000    nfezdn 58 dez846 dze 46z : https://stackoverflow.com/ , 8846 dlknzlncf"
parseAndRemoveStopWords(essai_string)
balise(parseAndRemoveStopWords(essai_string))



'DEBUTT sazoezdfpz numberrank vdvf fzf littlenumber bn numberrank de fdzenkfn bignumber datenumber littlenumber lol littlenumber dedez littlenumber kednzln phonenumber score score littlenumber score numberfour nfezdn littlenumber dezlittlenumber dze littlenumberz url numberfour dlknzlncf FINNN'

## Word count 

Dans cette partie, on va déterminer le vocabulaire de notre jeu d'entraînement ainsi que le nombre d'occurences par mot.

In [11]:
text_file = sc.textFile("D:/Projet-Spark/xaa_train_2000_ligne")
#print(type(text_file))
text_file= text_file.map(parseAndRemoveStopWords)
text_file= text_file.map(balise)

#text_file = sc.parallelize(text_file.collect())
#counts = text_file.flatMap(lambda line: line)  # line.split(" ")  si pas passer dans fonction
counts = text_file.flatMap(lambda line: line.split(" ")) \
            .map(lambda word : word.lower()) \
            .map(lambda word: (word, 1)) \
            .reduceByKey(lambda a, b: a + b)
            
             
        
#counts.take(5)
cou= counts.collect()

In [6]:
cou

[('debutt', 2000),
 ('us', 69),
 ('centers', 6),
 ('prevention', 5),
 ('school', 17),
 ('systems', 8),
 ('close', 15),
 ('outbreaks', 2),
 ('occurred', 4),
 ('reversed', 1),
 ('saying', 7),
 ('apparent', 1),
 ('mildness', 1),
 ('of', 1197),
 ('schools', 11),
 ('open', 21),
 ('even', 46),
 ('confirmed', 12),
 ('cases', 7),
 ('finnn', 2000),
 ('when', 94),
 ('winfrey', 2),
 ('somers', 1),
 ('share', 15),
 ('her', 76),
 ('controversial', 4),
 ('views', 4),
 ('bioidentical', 1),
 ('in', 943),
 ('numberfour', 144),
 ('rare', 6),
 ('dollop', 1),
 ('story', 5),
 ('titled', 1),
 ('crazy', 2),
 ('elk', 1),
 ('perfected', 1),
 ('long', 28),
 ('ago', 10),
 ('is', 378),
 ('now', 47),
 ('own', 13),
 ('fish', 3),
 ('ranked', 3),
 ('world', 34),
 ('littlenumber', 515),
 ('aces', 1),
 ('en', 3),
 ('route', 4),
 ('score', 48),
 ('win', 18),
 ('seventhseeded', 1),
 ('why', 10),
 ('everything', 4),
 ('have', 186),
 ('become', 12),
 ('king', 6),
 ('abdullah', 2),
 ('meet', 9),
 ('april', 7),
 ('stronger',

In [7]:
text_file = sc.textFile("D:/Projet-Spark/xaa_train_2000_ligne") # seulement les 10000 première lignes du document, car beaucoup trop long sinon 
print(type(text_file))
#print(type(text_file))
text_file= text_file.map(parseAndRemoveStopWords)
text_file.take(1000)

<class 'pyspark.rdd.RDD'>


['The US Centers for Disease Control and Prevention initially advised school systems to close if outbreaks occurred then reversed itself saying the apparent mildness of the virus meant most schools and day care centers should stay open even if they had confirmed cases of swine flu',
 'When Ms Winfrey invited Suzanne Somers to share her controversial views about bioidentical hormone treatment on her syndicated show in numberfour it won Ms Winfrey a rare dollop of unflattering press including a Newsweek cover story titled Crazy Talk Oprah Wacky Cures & You',
 'Elk calling a skill that hunters perfected long ago to lure game with the promise of a little romance is now its own sport',
 'Dont',
 'Fish ranked numberrank in the world fired littlenumber aces en route to a score score littlenumber littlenumber score littlenumber littlenumber win over seventhseeded Argentinian David Nalbandian',
 'Why does everything have to become such a big issue',
 'AMMAN Reuters King Abdullah of Jordan will 

In [8]:
from pyspark.sql.types import *
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
#df = SQLContext.createDataFrame(bigr, ["label", "valeur"])
word_count = sqlContext.createDataFrame(cou, ["Mot", "Occ_mot"])
word_count.show()

+----------+-------+
|       Mot|Occ_mot|
+----------+-------+
|    debutt|   2000|
|        us|     69|
|   centers|      6|
|prevention|      5|
|    school|     17|
|   systems|      8|
|     close|     15|
| outbreaks|      2|
|  occurred|      4|
|  reversed|      1|
|    saying|      7|
|  apparent|      1|
|  mildness|      1|
|        of|   1197|
|   schools|     11|
|      open|     21|
|      even|     46|
| confirmed|     12|
|     cases|      7|
|     finnn|   2000|
+----------+-------+
only showing top 20 rows



## Pos Tag (Word, tag) count 

On va ensuite tokeniser et attribuer une catégorie morphosyntaxique pour chaque mot du jeu d'entraînement.

In [69]:
from nltk import word_tokenize
from nltk import pos_tag
import nltk
def pos_tag(text):
    token = word_tokenize(text)
    t= nltk.pos_tag(token)
    s=""
    s="DEBUT"
    for word, tag in t :
            s=s+" "+tag
    s=s+" "+"FIN"
    return s 


#pos_tag("hello my name is Brian")

def balise(text):
    s="DEBUT"
    s= s+ " "+text+ " "+"FIN"
    return s

def mot1(s):
    return s[0]
def mot2(s):
    return s[1]

def mot3(s):
    return s[2]


In [10]:
pos_tag(parseAndRemoveStopWords( "On the opening of the trial , Mr. Taylor stayed in his cell ."))


'DEBUT IN DT NN IN DT NN NNP NNP VBD IN PRP$ NN FIN'

### Tag count

Pour chaque classe morphosyntaxique, on relève le nombre d'occurences.

In [11]:
text_file = sc.textFile("D:/Projet-Spark/xaa_train_2000_ligne")
#print(type(text_file))
text_file= text_file.map(parseAndRemoveStopWords)
text_file= text_file.map(pos_tag)

#text_file = sc.parallelize(text_file.collect())
#counts = text_file.flatMap(lambda line: line)  # line.split(" ")  si pas passer dans fonction
counts_tag = text_file.flatMap(lambda line: line.split(" ")) \
            .map(lambda word : word.lower()) \
            .map(lambda word: (word, 1)) \
            .reduceByKey(lambda a, b: a + b)
            
             
        
counts_tag.take(5)
pos_tag_count= counts_tag.collect()

In [12]:
from pyspark.sql.types import *
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
#df = SQLContext.createDataFrame(bigr, ["label", "valeur"])
tag_count = sqlContext.createDataFrame(pos_tag_count, ["tag", "Occ_tag"])
tag_count.show()

+----+-------+
| tag|Occ_tag|
+----+-------+
|nnps|    105|
|  in|   5575|
|  cc|   1432|
| vbd|   1943|
|  vb|   1548|
| vbn|   1148|
|  jj|   3259|
| rbs|     27|
| fin|   2000|
| wrb|    180|
|prp$|    608|
| vbz|    984|
| jjr|    210|
|  ex|     63|
|  cd|    406|
| vbp|    763|
|  wp|    179|
|   $|     77|
| wp$|      4|
|   #|      4|
+----+-------+
only showing top 20 rows



In [49]:
tag_count.toPandas().to_csv('tag_count.csv')

In [50]:
tag_count =sqlContext.read.csv(
    "tag_count.csv", header=True, mode="DROPMALFORMED"
)

from pyspark.sql.types import DoubleType

tag_count= tag_count.withColumn("Occ_tag", tag_count["Occ_tag"].cast(DoubleType()))
tag_count.show(4)

+---+----+-------+
|_c0| tag|Occ_tag|
+---+----+-------+
|  0|nnps|  105.0|
|  1|  in| 5575.0|
|  2|  cc| 1432.0|
|  3| vbd| 1943.0|
+---+----+-------+
only showing top 4 rows



### Transition tag count

On va ensuite relever le nombre d'occurences pour chaque transition de tags dans le jeu d'entraînement.

In [13]:
text_file = sc.textFile("D:/Projet-Spark/xaa_train_2000_ligne")
text_file= text_file.map(parseAndRemoveStopWords)
text_file= text_file.map(pos_tag)

bigrams_tag = text_file.flatMap(lambda line: line.split(".")) \
                .map(lambda word : word.lower())  \
                .map(lambda line: line.strip().split(" ")) \
                .flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:])))

pos_tag_trans= bigrams_tag.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
pos_tag_trans= pos_tag_trans.collect()
pos_tag_trans

[(('debut', 'dt'), 442),
 (('dt', 'nnp'), 574),
 (('nnps', 'in'), 14),
 (('nnp', 'nnp'), 2030),
 (('nnp', 'rb'), 85),
 (('nn', 'nns'), 481),
 (('nns', 'to'), 126),
 (('vb', 'in'), 187),
 (('prp', 'vbg'), 7),
 (('vbg', 'dt'), 188),
 (('dt', 'nn'), 2170),
 (('vbd', 'rbs'), 1),
 (('nn', 'nn'), 950),
 (('nns', 'md'), 96),
 (('rb', 'rb'), 128),
 (('vbd', 'vbn'), 209),
 (('nnp', 'to'), 51),
 (('to', 'nn'), 39),
 (('prp$', 'jj'), 134),
 (('in', 'jj'), 488),
 (('in', 'prp$'), 249),
 (('nnp', 'dt'), 154),
 (('nn', 'vbg'), 117),
 (('debut', 'nnp'), 478),
 (('nnp', 'vbg'), 61),
 (('nn', 'wdt'), 88),
 (('wdt', 'nns'), 2),
 (('rb', 'to'), 55),
 (('debut', 'nn'), 64),
 (('jj', 'vbd'), 8),
 (('vbd', 'jj'), 130),
 (('nn', 'to'), 312),
 (('to', 'dt'), 179),
 (('nnp', 'nn'), 541),
 (('jj', 'jj'), 222),
 (('wrb', 'vbz'), 3),
 (('vb', 'jj'), 137),
 (('nnp', 'md'), 60),
 (('nnp', 'nns'), 153),
 (('jjr', 'jj'), 14),
 (('vbg', 'nn'), 123),
 (('vbd', 'in'), 260),
 (('debut', 'to'), 14),
 (('vb', 'vb'), 16),
 

### P(tag2|tag1)

On peut ainsi déterminer une probabilité d'obtenir la classe i+1 sachant la classe i grâce aux statistiques effectuées sur notre jeu d'entraînement.

P(tag2|tag1 ) = Count(tag1,tag2)/ Count(tag1)

In [23]:
import pyspark.sql.functions as F

from pyspark.sql.types import *
sqlContext = SQLContext(sc)
from pyspark.sql import SQLContext
#df = SQLContext.createDataFrame(bigr, ["label", "valeur"])
df_tag = sqlContext.createDataFrame(pos_tag_trans, ["Pair", "valeur"])


from pyspark.sql.functions import lit

from pyspark.sql.functions import udf

mot1f = udf(mot1, StringType())
mot2f = udf(mot2, StringType())
df_tag= df_tag.select("Pair", "valeur")
df_tag1= df_tag.withColumn('tag1',mot1f(df_tag['Pair']))
df_tag1= df_tag1.withColumn('tag2',mot2f(df_tag['Pair']))
df_tag1 = df_tag1.withColumn("P(tag2|tag1)", lit(0))


### jointure tag_count et pair_tag_count
df_tag1 = df_tag1.join(tag_count, df_tag1.tag1 ==tag_count.tag)

df_tag2 =df_tag1.withColumn("P(tag2|tag1)", (F.col("valeur") / F.col("Occ_tag")))
df_tag2 =df_tag2.drop("tag").drop("Occ_tag").drop("valeur").drop("Pair")
df_tag2.show()



+----+----+--------------------+
|tag1|tag2|        P(tag2|tag1)|
+----+----+--------------------+
|  cc|prp$| 0.02723463687150838|
|  cc|  jj| 0.11731843575418995|
|  cc|  ex|0.004888268156424581|
|  cc|  in|0.046089385474860335|
|  cc|  vb| 0.05307262569832402|
|  cc| wrb|0.003491620111731...|
|  cc| vbz| 0.02653631284916201|
|  cc| vbd|0.061452513966480445|
|  cc| jjr|0.013966480446927373|
|  cc| vbp|0.006983240223463687|
|  cc|nnps|0.002094972067039106|
|  cc| vbn|0.019553072625698324|
|  cc|  cd|  0.0111731843575419|
|  cc|  wp|0.003491620111731...|
|  cc|   $|0.001396648044692...|
|  cc|  cc|6.983240223463687E-4|
|  cc| nnp| 0.13687150837988826|
|  cc|  nn|  0.1180167597765363|
|  cc| prp| 0.05377094972067039|
|  cc|  dt| 0.12639664804469275|
+----+----+--------------------+
only showing top 20 rows



In [24]:
df_tag2.toPandas().to_csv('trans_tag.csv')

In [25]:
trans_tag =sqlContext.read.csv(
    "trans_tag.csv", header=True, mode="DROPMALFORMED"
)

from pyspark.sql.types import DoubleType

trans_tag = trans_tag.withColumn("P(tag2|tag1)", trans_tag["P(tag2|tag1)"].cast(DoubleType()))
trans_tag.show(4)

+---+----+----+--------------------+
|_c0|tag1|tag2|        P(tag2|tag1)|
+---+----+----+--------------------+
|  0|  cc|prp$| 0.02723463687150838|
|  1|  cc|  jj| 0.11731843575418995|
|  2|  cc|  ex|0.004888268156424581|
|  3|  cc|  in|0.046089385474860335|
+---+----+----+--------------------+
only showing top 4 rows



## Pair Word Count

De même que précédemment, on va relever le nombre d'occurences pour chaque mot dans le jeu d'entraînement.

In [26]:
text_file = sc.textFile("D:/Projet-Spark/xaa_train_2000_ligne")
text_file= text_file.map(parseAndRemoveStopWords)
text_file= text_file.map(balise)


bigrams = text_file.flatMap(lambda line: line.split(".")) \
                .map(lambda word : word.lower())  \
                .map(lambda line: line.strip().split(" ")) \
                .flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:])))

e= bigrams.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

In [27]:
e.take(10)

[(('debut', 'the'), 287),
 (('us', 'centers'), 1),
 (('for', 'disease'), 2),
 (('disease', 'control'), 2),
 (('control', 'and'), 2),
 (('initially', 'advised'), 1),
 (('school', 'systems'), 1),
 (('outbreaks', 'occurred'), 1),
 (('apparent', 'mildness'), 1),
 (('mildness', 'of'), 1)]

In [28]:
bigr = e.collect()

In [29]:
from pyspark.sql.types import *
sqlContext = SQLContext(sc)
from pyspark.sql import SQLContext
#df = SQLContext.createDataFrame(bigr, ["label", "valeur"])
df = sqlContext.createDataFrame(bigr, ["Pair", "valeur"])
df.show()


+--------------------+------+
|                Pair|valeur|
+--------------------+------+
|        [debut, the]|   287|
|       [us, centers]|     1|
|      [for, disease]|     2|
|  [disease, control]|     2|
|      [control, and]|     2|
|[initially, advised]|     1|
|   [school, systems]|     1|
|[outbreaks, occur...|     1|
|[apparent, mildness]|     1|
|      [mildness, of]|     1|
|        [the, virus]|     3|
|      [virus, meant]|     1|
|       [meant, most]|     1|
|          [and, day]|     1|
|         [day, care]|     1|
|      [should, stay]|     1|
|        [open, even]|     1|
|          [if, they]|     4|
|         [they, had]|     4|
|  [confirmed, cases]|     1|
+--------------------+------+
only showing top 20 rows



In [30]:
def mot1(s):
    return s[0]
def mot2(s):
    return s[1]

In [31]:
from pyspark.sql.functions import lit

from pyspark.sql.functions import udf
mot1f = udf(mot1, StringType())
mot2f = udf(mot2, StringType())
df= df.select("Pair", "valeur")
df1= df.withColumn('mot1',mot1f(df['Pair']))
df1= df1.withColumn('mot2',mot2f(df['Pair']))
df1 = df1.withColumn("P(mot2|mot1)", lit(0))


In [32]:
#df1.toPandas().to_csv('pair_word_count1k.csv')
#word_count.toPandas().to_csv('word_count1k')

##  Jointure Pair word_count et word_count

On va ici rassembler les informations précédentes: occurence des mots et occurence des paires de mots afin de calculer la probabilité d'obtenir le mot i+1 sachant le mot i grâce aux statistiques effectuées sur le jeu d'entraînement.

On réutilise les deux tables Pair Word_count et Word_Count:

In [33]:
df2 = df1
df2 = df2.join(word_count, df2.mot1 ==word_count.Mot)

df2.show(5)

+----------------+------+----+---------+------------+---+-------+
|            Pair|valeur|mot1|     mot2|P(mot2|mot1)|Mot|Occ_mot|
+----------------+------+----+---------+------------+---+-------+
|    [art, crawl]|     1| art|    crawl|           0|art|      9|
|[art, galleries]|     1| art|galleries|           0|art|      9|
|  [art, classes]|     1| art|  classes|           0|art|      9|
|      [art, one]|     1| art|      one|           0|art|      9|
|      [art, and]|     1| art|      and|           0|art|      9|
+----------------+------+----+---------+------------+---+-------+
only showing top 5 rows



In [34]:
import pyspark.sql.functions as F

df3 =df2.withColumn("P(mot2|mot1)", (F.col("valeur") / F.col("Occ_mot")))
df3.show()

+------------------+------+---------+---------+------------------+---------+-------+
|              Pair|valeur|     mot1|     mot2|      P(mot2|mot1)|      Mot|Occ_mot|
+------------------+------+---------+---------+------------------+---------+-------+
|      [art, crawl]|     1|      art|    crawl|0.1111111111111111|      art|      9|
|  [art, galleries]|     1|      art|galleries|0.1111111111111111|      art|      9|
|    [art, classes]|     1|      art|  classes|0.1111111111111111|      art|      9|
|        [art, one]|     1|      art|      one|0.1111111111111111|      art|      9|
|        [art, and]|     1|      art|      and|0.1111111111111111|      art|      9|
|         [art, in]|     2|      art|       in|0.2222222222222222|      art|      9|
|     [art, school]|     1|      art|   school|0.1111111111111111|      art|      9|
|      [art, world]|     1|      art|    world|0.1111111111111111|      art|      9|
| [aruba, wireless]|     1|    aruba| wireless|               1.0

## P(mot2|mot1)

On calcule la probabilité d'obtenir le mot i+1 sachant le mot i:

P(mot2|mot1) = Count(mot1,mot2)/Count(mot1)

In [35]:
import pyspark.sql.functions as F

df4 =df3.drop("Mot").drop("Occ_Mot").drop("valeur").drop("Pair")
df4.show()


+---------+---------+------------------+
|     mot1|     mot2|      P(mot2|mot1)|
+---------+---------+------------------+
|      art|    crawl|0.1111111111111111|
|      art|galleries|0.1111111111111111|
|      art|  classes|0.1111111111111111|
|      art|      one|0.1111111111111111|
|      art|      and|0.1111111111111111|
|      art|       in|0.2222222222222222|
|      art|   school|0.1111111111111111|
|      art|    world|0.1111111111111111|
|    aruba| wireless|               1.0|
|  barrier|      fin|               0.5|
|  barrier|       to|               0.5|
|   binghi| thompson|               1.0|
|   biting|      off|               1.0|
|  bonobos|      the|               1.0|
|   brands|    today|               0.5|
|   brands|     such|               0.5|
|    bushs|firstrate|               1.0|
|carbonate|      fin|               1.0|
| cautious|       at|               0.5|
| cautious|    about|               0.5|
+---------+---------+------------------+
only showing top

## Accès aux valeurs 

In [36]:
v =df4.where(df4.mot1=='arts').where(df4.mot2=='center').select('P(mot2|mot1)').collect()

In [37]:
v[0][0]

0.18181818181818182

## Enregistrer la base 


In [38]:

df4.toPandas().to_csv('transition2000.csv')




In [29]:
#word_count_first1.toPandas().to_csv('Initiation100000')
#word_count_last1.toPandas().to_csv('Last100000')

In [39]:

word_count.toPandas().to_csv('word_count2000')

In [4]:
trans =sqlContext.read.csv(
    "transition2000.csv", header=True, mode="DROPMALFORMED"
)

from pyspark.sql.types import DoubleType

trans = trans.withColumn("P(mot2|mot1)", trans["P(mot2|mot1)"].cast(DoubleType()))

In [41]:
trans.show(4)

+---+----+---------+------------------+
|_c0|mot1|     mot2|      P(mot2|mot1)|
+---+----+---------+------------------+
|  0| art|    crawl|0.1111111111111111|
|  1| art|galleries|0.1111111111111111|
|  2| art|  classes|0.1111111111111111|
|  3| art|      one|0.1111111111111111|
+---+----+---------+------------------+
only showing top 4 rows



In [3]:
word_count =sqlContext.read.csv(
    "word_count2000", header=True, mode="DROPMALFORMED"
)

from pyspark.sql.types import DoubleType

word_count = word_count.withColumn("Occ_mot", word_count["Occ_mot"].cast(DoubleType()))
word_count.show(4)

+---+----------+-------+
|_c0|       Mot|Occ_mot|
+---+----------+-------+
|  0|    debutt| 2000.0|
|  1|        us|   69.0|
|  2|   centers|    6.0|
|  3|prevention|    5.0|
+---+----------+-------+
only showing top 4 rows



# Utilisation du Modèle Bi-gramme

## Faire une prédiction du mot manquant  :

On va tout d'abord prédire l'endroit/l'indice du mot manquant puis déterminer le mot à insérer à partir du jeu d'entraînement.

### Determiner l'indice : argmin de P(tag2|tag1) 

In [128]:
import numpy as np
def indice_mot_manquant(string,trans_tag ):
    i=0 
    s=string.split(' ')
    string = pos_tag(string)
    string =string.lower()
    miss = string.split(' ')
    print(miss)
    vecteur_prob=[]
    for j in range(len(miss)-1) : 
        
        
        v =trans_tag.where(trans_tag.tag1==miss[j]).where(trans_tag.tag2==miss[j+1]).select('P(tag2|tag1)').collect()
        if v==[] :

            vecteur_prob.append(0)
        else:
            vecteur_prob.append(v[0][0] )
    print(vecteur_prob)
    indice =np.argmin(vecteur_prob)
    print("mot manquant prédit  entre  : " ,s[indice-1], " et " , s[indice])  # car on a rajouté la balise au début
    print("tag mot entourant ", miss[indice]," ", miss[indice+1] )
    tag1= miss[indice]
    tag2= miss[indice+1]
    return np.argmin(vecteur_prob), tag1 , tag2




In [129]:
#worshippers said they were   : indice du mot manquant => 7 : mot manquant entre worshippers et they 
string ='at the targeted sites on thursday worshippers they were relieved that the plot was halted but they didnt intend to change their daily routines'

indice_mot_manquant(string,trans_tag )

['debut', 'in', 'dt', 'jj', 'nns', 'in', 'jj', 'nns', 'prp', 'vbd', 'vbn', 'in', 'dt', 'nn', 'vbd', 'vbn', 'cc', 'prp', 'vbp', 'vbp', 'to', 'vb', 'prp$', 'jj', 'nns', 'fin']
[0.101, 0.33955156950672644, 0.2259552042160738, 0.2086529610309911, 0.26162982514021776, 0.08753363228699551, 0.2086529610309911, 0.012207192345760474, 0.3398576512455516, 0.10756562017498714, 0.3806620209059233, 0.33955156950672644, 0.47650417215634605, 0.058379888268156425, 0.10756562017498714, 0.025261324041811847, 0.05377094972067039, 0.1797153024911032, 0.01834862385321101, 0.06815203145478375, 0.6610032362459547, 0.040051679586563305, 0.22039473684210525, 0.2086529610309911, 0.11778290993071594]
mot manquant prédit  entre  :  worshippers  et  they
tag mot entourant  nns   prp


(7, 'nns', 'prp')

Ici le mot manquant se trouve à l'indice 7.

### Prédiction du mot manquant, après détermination de l'indice : argmax P(test|mot1)* p(mot2|test) 

Afin de prédire le mot manquant à insérer, on va calculer la probabilité suivante P(test|mot1)* p(mot2|test) pour chaque mot 'test' du vocabulaire du jeu d'entraînement et récupérer le mot ayant la plus grande probabilité.

La condition est d'avoir les 3 mots : mot1, test, mot2 qui sont déjà apparus dans le training set et de garder les mots tests tels que leur probabilité P(test|mot1)* p(mot2|test) est supérieur à zéro.

Par rapport au code,
Dans le train set, on a vecteur prob = 0 si la suite de mot dans le test : mot1, test, mot2 pour tous les mots testés appartenant au vocabulaire du train n'existe pas.

Si le modèle n'est pas assez entrainé, c'est à dire ayant surtout peu de données d'entraînement, on aura un vecteur de probabilité fréquemment nul par manque de vocabulaire.
On se contentera d'une prédiction du tag du mot manquant P(test_tag|tag1)* p(tag2|test_tag) 

In [132]:
def predire_mot_manquant(indice_mot_manquant, word_count, string):
    string = balise(string)
    miss = string.split(' ')
    vecteur_mot_prob =[]
   
    eee = word_count.select('Mot').collect()  # liste mot du vocabulaire, du corpus
    print(len(eee))
    
    print(len(miss))
 
    # mot  manquant dans la phrase 
    
    
    print( indice_mot_manquant)
    inter_mot = miss[indice_mot_manquant:indice_mot_manquant+2]
    mot1 = inter_mot[0]
    mot2 =inter_mot[1]
    print(mot1)
    print(mot2)

    for i in range(len(eee)):
        mot_test = eee[i][0]
        #print(mot_test)

        v1 =trans.where(trans.mot1==mot1).where(trans.mot2==mot_test).select('P(mot2|mot1)').collect()
        v2 = trans.where(trans.mot1==mot_test).where(trans.mot2==mot2).select('P(mot2|mot1)').collect()
        #print(v1)
        #print(v2)

        if v1==[] or v2 ==[] :
            vecteur_mot_prob.append(0)
        else :
            vecteur_mot_prob.append(v1[0][0]*v2[0][0] )
            
            
    # Si le vecteur est nul, donc si le modèle ne peut determiner un mot possible juste pour les phrases du test set 
    if np.sum(vecteur_mot_prob)==0 :
        print("impossible de prédire le mot => prédire le tag du mot manquant?")
        return "dommage "
  
        
    indice_mot = np.argmax(vecteur_mot_prob)
    print("mot prédit " , eee[indice_mot][0])
    return eee[indice_mot][0] , vecteur_mot_prob



#worshippers said they were  : le mot à prédire est 'said' à la position 7 
string ='at the targeted sites on thursday worshippers they were relieved that the plot was halted but they didnt intend to change their daily routines'
i_m =indice_mot_manquant(string,trans_tag )[0]
print("indice du mot manquant ", i_m)
mot_manquant_pred = predire_mot_manquant(i_m, word_count, string)


['debut', 'in', 'dt', 'jj', 'nns', 'in', 'jj', 'nns', 'prp', 'vbd', 'vbn', 'in', 'dt', 'nn', 'vbd', 'vbn', 'cc', 'prp', 'vbp', 'vbp', 'to', 'vb', 'prp$', 'jj', 'nns', 'fin']
[0.101, 0.33955156950672644, 0.2259552042160738, 0.2086529610309911, 0.26162982514021776, 0.08753363228699551, 0.2086529610309911, 0.012207192345760474, 0.3398576512455516, 0.10756562017498714, 0.3806620209059233, 0.33955156950672644, 0.47650417215634605, 0.058379888268156425, 0.10756562017498714, 0.025261324041811847, 0.05377094972067039, 0.1797153024911032, 0.01834862385321101, 0.06815203145478375, 0.6610032362459547, 0.040051679586563305, 0.22039473684210525, 0.2086529610309911, 0.11778290993071594]
mot manquant prédit  entre  :  worshippers  et  they
tag mot entourant  nns   prp
indice du mot manquant  7
9689
26
7
worshippers
they
mot prédit  said


In [143]:
# Si le vecteur est nul, donc si le modèle ne peut determiner un mot
    # car par exemple  modèle pas assez entrainer donc pas assez de vocabulaire, on peut renvoyer le tag du mot manquant 

def predire_tag(indice_mot_manquant, word_count, string, tag_count,tag1, tag2):
    vecteur_tag_prob =[]
    tags = tag_count.select('tag').collect()

    for j in range(len(tags)):
        tag_test = tags[j][0]
        t1 =trans_tag.where(trans_tag.tag1==tag1).where(trans_tag.tag2==tag_test).select('P(tag2|tag1)').collect()
        t2 = trans_tag.where(trans_tag.tag1==tag_test).where(trans_tag.tag2==tag2).select('P(tag2|tag1)').collect()

    if t1==[] or t2 ==[] :
        vecteur_tag_prob.append(0)
    else :
        vecteur_tag_prob.append(t1[0][0]*t2[0][0] )

    indice_tag = np.argmax(vecteur_tag_prob)
    print("tag prédit du mot manquant : " , tags[indice_tag][0])
    #return tags[indice_tag][0] , vecteur_tag_prob
    return tags[indice_tag][0]
    
    

# Faire une prédiction sur le test 

On nettoie préalablement les phrases du jeu de test comme ce que l'on a fait sur le train, et on y ajoute les balises.

In [44]:
test_text_file = sc.textFile("D:/Projet-Spark/test_v2.txt")
#print(type(text_file))
#test_text_file.map(parseAndRemoveStopWords).take(5)
test = test_text_file.take(20)
test


['"id","sentence"',
 '1,"He added that people should not mess with mother nature , and let sharks be ."',
 '2,"The Great Britain team bus will trundle along the Via dei Gladiatori in Rome tomorrow morning deliver two battlers to the arena that is the World Championship venue ."',
 '3,"Investment return and principal value of an investment in a variable annuity will fluctuate , so units , when redeemed , may be worth more or less than their cost ."',
 '4,"The \'s bloody body was discovered on a bed ."',
 '5,"Her adds that most Americans "" want to be seen in their big house with a big car . """',
 '6,"Michael Jackson could be forced to fly to the High Court in London to testify in a case being brought against him the King of Bahrain \'s son ."',
 '7,"The Wizards recovered from a 4-9 start season , and several of the team \'s key players have been around long enough to know that a bad start does not necessarily lead to a bad finish ."',
 '8,""" People fears appear to be racing ahead of r

### Test sur la phrase 17 du test_set

In [8]:
phrase_trois = "On the opening of the trial , Mr. Taylor stayed in his cell ."
phrase_trois_clean = parseAndRemoveStopWords(phrase_trois)
phrase_trois_clean

'On the opening of the trial Mr Taylor stayed in his cell'

In [138]:
indice , tag1 , tag2 = indice_mot_manquant(phrase_trois_clean,trans_tag )
indice

['debut', 'in', 'dt', 'nn', 'in', 'dt', 'nn', 'nnp', 'nnp', 'vbd', 'in', 'prp$', 'nn', 'fin']
[0.101, 0.33955156950672644, 0.47650417215634605, 0.27849162011173184, 0.33955156950672644, 0.47650417215634605, 0.04064245810055866, 0.38345296562145825, 0.09387986399697772, 0.13381369016984046, 0.04466367713004484, 0.4786184210526316, 0.11997206703910615]
mot manquant prédit  entre  :  trial  et  Mr
tag mot entourant  nn   nnp


6

### Remarques:

Si on veut avoir une prédiction de mot avec notre modèle il faut que la suite de mots: mot1, mot_test, mot2 soit déja apparu dans le train. 
L'algorithme prédit mot_test en regardant tous les mots du vocabulaire et sélectionne ceux qui ont P(mot_test|mot1)*P(mot2|mot_test)>0
Plus notre jeu d'entraînement est grand et plus notre modèle est puissant.
L'inconvénient est le temps de prédiction qui grandit avec la taille du jeu d'entraînement. Car l'algorithme teste tous les mots du vocabulaire.

In [12]:
import numpy as np
string = phrase_trois_clean
test_mot_pred = predire_mot_manquant(6, word_count, string)

9689
14
6
trial
Mr
debutt
[]
[]
us
[]
[]
centers
[]
[]
prevention
[]
[]
school
[]
[]
systems
[]
[]
close
[]
[]
outbreaks
[]
[]
occurred
[]
[]
reversed
[]
[]
saying
[]
[]
apparent
[]
[]
mildness
[]
[]
of
[]
[]
schools
[]
[]
open
[]
[]
even
[]
[]
confirmed
[]
[]
cases
[]
[]
finnn
[]
[]
when
[]
[]
winfrey
[]
[]
somers
[]
[]
share
[]
[]
her
[]
[]
controversial
[]
[]
views
[]
[]
bioidentical
[]
[]
in
[Row(P(mot2|mot1)=0.25)]
[]
numberfour
[]
[]
rare
[]
[]
dollop
[]
[]
story
[]
[]
titled
[]
[]
crazy
[]
[]
elk
[]
[]
perfected
[]
[]
long
[]
[]
ago
[]
[]
is
[]
[]
now
[]
[]
own
[]
[]
fish
[]
[]
ranked
[]
[]
world
[]
[]
littlenumber
[]
[]
aces
[]
[]
en
[]
[]
route
[]
[]
score
[]
[]
win
[]
[]
seventhseeded
[]
[]
why
[]
[]
everything
[]
[]
have
[]
[]
become
[]
[]
king
[]
[]
abdullah
[]
[]
meet
[]
[]
april
[]
[]
stronger
[]
[]
middle
[]
[]
east
[]
[]
peacemaking
[]
[]
palace
[]
[]
officials
[]
[]
said
[]
[]
help
[]
[]
traffic
[]
[]
charge
[]
[]
tfl
[]
[]
suspending
[]
[]
road
[]
[]
major
[]
[]
roads

[]
[]
size
[]
[]
pinzolo
[]
[]
italy
[]
[]
ap
[]
[]
quitting
[]
[]
confirmation
[]
[]
reappointment
[]
[]
domenech
[]
[]
rental
[]
[]
attend
[]
[]
midday
[]
[]
christmas
[]
[]
meal
[]
[]
newspaper
[]
[]
intense
[]
[]
regarded
[]
[]
banking
[]
[]
naive
[]
[]
briefings
[]
[]
tories
[]
[]
creating
[]
[]
points
[]
[]
rockets
[]
[]
lifted
[]
[]
tsunami
[]
[]
rolled
[]
[]
argued
[]
[]
encounter
[]
[]
zerocarbon
[]
[]
performance
[]
[]
indications
[]
[]
stabilising
[]
[]
cautious
[]
[]
stage
[]
[]
ubs
[]
[]
swiss
[]
[]
badly
[]
[]
hurt
[]
[]
carnage
[]
[]
mortgage
[]
[]
juergen
[]
[]
german
[]
[]
kong
[]
[]
losing
[]
[]
cannot
[]
[]
trips
[]
[]
hamas
[]
[]
tuesday
[]
[]
leaders
[]
[]
discussing
[]
[]
israeli
[]
[]
soldier
[]
[]
gilad
[]
[]
thirty
[]
[]
forced
[]
[]
america
[]
[]
roman
[]
[]
los
[]
[]
angeles
[]
[]
dismissed
[]
[]
believed
[]
[]
limited
[]
[]
africas
[]
[]
rift
[]
[]
valley
[]
[]
region
[]
[]
pits
[]
[]
heart
[]
[]
short
[]
[]
kilimanjaro
[]
[]
barrell
[]
[]
chancellor
[]
[]
r

[]
[]
im
[]
[]
certainly
[]
[]
spendulous
[]
[]
bailouts
[]
[]
lineup
[]
[]
goes
[]
[]
computers
[]
[]
include
[]
[]
cameras
[]
[]
cookware
[]
[]
electric
[]
[]
customer
[]
[]
used
[]
[]
hate
[]
[]
carson
[]
[]
latenight
[]
[]
university
[]
[]
stanislaus
[]
[]
contract
[]
[]
sarah
[]
[]
speech
[]
[]
recycling
[]
[]
susana
[]
[]
advancement
[]
[]
bonus
[]
[]
cap
[]
[]
fiveyear
[]
[]
term
[]
[]
promises
[]
[]
reform
[]
[]
governance
[]
[]
remembers
[]
[]
seeing
[]
[]
blob
[]
[]
watch
[]
[]
movies
[]
[]
antinato
[]
[]
demonstrators
[]
[]
summit
[]
[]
clashed
[]
[]
strasbourg
[]
[]
firing
[]
[]
bullets
[]
[]
youths
[]
[]
recruit
[]
[]
forces
[]
[]
feels
[]
[]
fridays
[]
[]
bounce
[]
[]
particular
[]
[]
contest
[]
[]
maritime
[]
[]
worked
[]
[]
known
[]
[]
larry
[]
[]
bird
[]
[]
mates
[]
[]
p
[]
[]
right
[]
[]
class
[]
[]
gerber
[]
[]
politics
[]
[]
prepare
[]
[]
glendale
[]
[]
frank
[]
[]
honored
[]
[]
cleanup
[]
[]
house
[]
[]
designed
[]
[]
above
[]
[]
temporarily
[]
[]
fear
[]
[]
decomp

[]
[]
pinkcheeked
[]
[]
les
[]
[]
outskirts
[]
[]
chamonix
[]
[]
vaulted
[]
[]
concerns
[]
[]
look
[]
[]
challenges
[]
[]
favorite
[]
[]
stressed
[]
[]
endorsing
[]
[]
thorn
[]
[]
interviews
[]
[]
magazines
[]
[]
emails
[]
[]
agent
[]
[]
loyal
[]
[]
egypt
[]
[]
actual
[]
[]
charlottes
[]
[]
fresnel
[]
[]
lighthouse
[]
[]
waiting
[]
[]
mouth
[]
[]
sixyearold
[]
[]
drinking
[]
[]
george
[]
[]
washingtons
[]
[]
wonder
[]
[]
burntout
[]
[]
derby
[]
[]
discomfort
[]
[]
maps
[]
[]
governments
[]
[]
horrors
[]
[]
revolution
[]
[]
decade
[]
[]
crushing
[]
[]
tiananmen
[]
[]
brave
[]
[]
thwarted
[]
[]
pry
[]
[]
gears
[]
[]
propaganda
[]
[]
machine
[]
[]
sellon
[]
[]
upton
[]
[]
brescia
[]
[]
claimed
[]
[]
becoming
[]
[]
summers
[]
[]
proms
[]
[]
fingers
[]
[]
ears
[]
[]
chick
[]
[]
european
[]
[]
registered
[]
[]
swaziland
[]
[]
castillo
[]
[]
stopped
[]
[]
law
[]
[]
language
[]
[]
attempted
[]
[]
cambodia
[]
[]
raged
[]
[]
cambodias
[]
[]
khmer
[]
[]
rouge
[]
[]
peasants
[]
[]
intellectuals
[]

[]
[]
mexican
[]
[]
mike
[]
[]
tamvakis
[]
[]
economics
[]
[]
predict
[]
[]
euros
[]
[]
assigns
[]
[]
enterprise
[]
[]
reoffending
[]
[]
nao
[]
[]
tended
[]
[]
hide
[]
[]
vision
[]
[]
freeview
[]
[]
channels
[]
[]
library
[]
[]
ondemand
[]
[]
programming
[]
[]
impala
[]
[]
butterscotch
[]
[]
peanut
[]
[]
scrinium
[]
[]
gold
[]
[]
successfully
[]
[]
clearance
[]
[]
asylum
[]
[]
tribunal
[]
[]
inquiry
[]
[]
partially
[]
[]
trolls
[]
[]
paste
[]
[]
await
[]
[]
policies
[]
[]
enduring
[]
[]
coastline
[]
[]
canals
[]
[]
marshes
[]
[]
leaves
[]
[]
brunt
[]
[]
tracking
[]
[]
abortion
[]
[]
sept
[]
[]
hernandez
[]
[]
byrd
[]
[]
seacrest
[]
[]
viewers
[]
[]
versus
[]
[]
gunmen
[]
[]
militarycom
[]
[]
halted
[]
[]
michele
[]
[]
bright
[]
[]
poised
[]
[]
country
[]
[]
eight
[]
[]
thick
[]
[]
african
[]
[]
indicate
[]
[]
consumers
[]
[]
position
[]
[]
feed
[]
[]
dates
[]
[]
tournament
[]
[]
involving
[]
[]
wisely
[]
[]
based
[]
[]
petit
[]
[]
executives
[]
[]
beatles
[]
[]
xbox
[]
[]
proceeds
[]
[

...

[]
[]
legs
[]
[]
marine
[]
[]
reserves
[]
[]
southcentral
[]
[]
portions
[]
[]
balked
[]
[]
announcement
[]
[]
recession
[]
[]
eighties
[]
[]
teens
[]
[]
fiver
[]
[]
hank
[]
[]
kindred
[]
[]
continuation
[]
[]
status
[]
[]
quo
[]
[]
reforms
[]
[]
gingerly
[]
[]
nationals
[]
[]
southampton
[]
[]
hampshire
[]
[]
serious
[]
[]
organised
[]
[]
gotten
[]
[]
heavier
[]
[]
powerful
[]
[]
retailing
[]
[]
auchan
[]
[]
hired
[]
[]
toys
[]
[]
octopus
[]
[]
squeaking
[]
[]
everyday
[]
[]
sides
[]
[]
soft
[]
[]
flakes
[]
[]
adhere
[]
[]
defects
[]
[]
hornets
[]
[]
hilton
[]
[]
chandler
[]
[]
ankle
[]
[]
nuggets
[]
[]
denvers
[]
[]
thirdquarter
[]
[]
advertisements
[]
[]
capability
[]
[]
embed
[]
[]
shoot
[]
[]
nosesay
[]
[]
latch
[]
[]
kasbah
[]
[]
toubkal
[]
[]
bransons
[]
[]
imlil
[]
[]
yusuf
[]
[]
upright
[]
[]
bed
[]
[]
aides
[]
[]
f
[]
[]
dripping
[]
[]
taps
[]
[]
waste
[]
[]
bath
[]
[]
ravi
[]
[]
unable
[]
[]
train
[]
[]
stomach
[]
[]
complaint
[]
[]
leftarm
[]
[]
irg
[]
[]
approached
[]
[]
p

[]
[]
mesh
[]
[]
clutch
[]
[]
buycom
[]
[]
kicking
[]
[]
doors
[]
[]
blair
[]
[]
knife
[]
[]
neighbour
[]
[]
allegations
[]
[]
gulf
[]
[]
explosions
[]
[]
mayons
[]
[]
steep
[]
[]
sounded
[]
[]
rat
[]
[]
romney
[]
[]
flicked
[]
[]
scrapbook
[]
[]
containing
[]
[]
unpublished
[]
[]
photos
[]
[]
che
[]
[]
nancy
[]
[]
siena
[]
[]
donations
[]
[]
lockton
[]
[]
contributions
[]
[]
patience
[]
[]
capitol
[]
[]
incomes
[]
[]
aid
[]
[]
rainfall
[]
[]
deficit
[]
[]
crop
[]
[]
invaluable
[]
[]
orrey
[]
[]
amusement
[]
[]
association
[]
[]
gillian
[]
[]
griffiths
[]
[]
connie
[]
[]
trucks
[]
[]
bangkok
[]
[]
isa
[]
[]
censor
[]
[]
inca
[]
[]
selfpublishing
[]
[]
propofol
[]
[]
diprivan
[]
[]
imply
[]
[]
aware
[]
[]
exotic
[]
[]
foreignness
[]
[]
creditors
[]
[]
chrysler
[]
[]
remembered
[]
[]
vinegar
[]
[]
fast
[]
[]
increases
[]
[]
advocates
[]
[]
blanket
[]
[]
piece
[]
[]
lauren
[]
[]
laverne
[]
[]
furthermore
[]
[]
civilians
[]
[]
reluctantly
[]
[]
advance
[]
[]
cashiers
[]
[]
belittled
[]
[]


[]
[]
coaxes
[]
[]
astounding
[]
[]
actors
[]
[]
motorcyclist
[]
[]
greencoloured
[]
[]
kawasaki
[]
[]
pillion
[]
[]
collision
[]
[]
propose
[]
[]
legitimate
[]
[]
lobbyist
[]
[]
downplayed
[]
[]
latin
[]
[]
epitaph
[]
[]
swears
[]
[]
cullen
[]
[]
lone
[]
[]
ana
[]
[]
winds
[]
[]
burn
[]
[]
freeway
[]
[]
updated
[]
[]
bst
[]
[]
geography
[]
[]
ha
[]
[]
bedford
[]
[]
user
[]
[]
allowance
[]
[]
badenoch
[]
[]
clark
[]
[]
kaohsiung
[]
[]
scolding
[]
[]
favourite
[]
[]
bauge
[]
[]
repechages
[]
[]
malaysian
[]
[]
awang
[]
[]
crampton
[]
[]
directly
[]
[]
erotic
[]
[]
exboyfriend
[]
[]
actuary
[]
[]
teammates
[]
[]
embargo
[]
[]
infuriated
[]
[]
bomber
[]
[]
afghans
[]
[]
johan
[]
[]
shootout
[]
[]
overtime
[]
[]
solidify
[]
[]
policymakers
[]
[]
gs
[]
[]
likelihood
[]
[]
precedent
[]
[]
siphon
[]
[]
ren
[]
[]
nutr
[]
[]
ipl
[]
[]
governing
[]
[]
longstalled
[]
[]
commenee
[]
[]
shut
[]
[]
athletics
[]
[]
sanctions
[]
[]
imposed
[]
[]
electoral
[]
[]
suppression
[]
[]
easytoread
[]
[]
barom

[]
[]
forgotten
[]
[]
populations
[]
[]
opposed
[]
[]
svengran
[]
[]
reacted
[]
[]
angrily
[]
[]
accusations
[]
[]
circus
[]
[]
engulfed
[]
[]
projector
[]
[]
pedestal
[]
[]
transient
[]
[]
aura
[]
[]
hadnt
[]
[]
hosting
[]
[]
pool
[]
[]
ulster
[]
[]
pace
[]
[]
unveiled
[]
[]
provider
[]
[]
automation
[]
[]
brands
[]
[]
adwords
[]
[]
downturn
[]
[]
flydubai
[]
[]
announcements
[]
[]
boeing
[]
[]
shorttomediumrange
[]
[]
staple
[]
[]
responded
[]
[]
variety
[]
[]
sos
[]
[]
fishing
[]
[]
whales
[]
[]
diet
[]
[]
adequate
[]
[]
marketplace
[]
[]
adult
[]
[]
magistrate
[]
[]
changsan
[]
[]
khatamis
[]
[]
pull
[]
[]
thankful
[]
[]
jonathan
[]
[]
arghandab
[]
[]
disputed
[]
[]
reelection
[]
[]
batch
[]
[]
hauled
[]
[]
vienna
[]
[]
austrian
[]
[]
waited
[]
[]
divine
[]
[]
promising
[]
[]
garrido
[]
[]
antonio
[]
[]
superbly
[]
[]
bluechip
[]
[]
circles
[]
[]
trespassing
[]
[]
sellek
[]
[]
fatwas
[]
[]
twostate
[]
[]
sa
[]
[]
wave
[]
[]
lifetime
[]
[]
yomiuri
[]
[]
japans
[]
[]
esteemed
[]
[]
y

[]
[]
lehtonen
[]
[]
vasicek
[]
[]
fashioned
[]
[]
whats
[]
[]
nearest
[]
[]
expressed
[]
[]
workplace
[]
[]
cr
[]
[]
weighing
[]
[]
disburse
[]
[]
patersons
[]
[]
murky
[]
[]
punk
[]
[]
menace
[]
[]
ramalama
[]
[]
thrill
[]
[]
dynamic
[]
[]
ensued
[]
[]
recover
[]
[]
tornado
[]
[]
accelerated
[]
[]
techcrunch
[]
[]
zynga
[]
[]
destiny
[]
[]
retirees
[]
[]
pouring
[]
[]
bottle
[]
[]
glen
[]
[]
jerba
[]
[]
hartlepool
[]
[]
strengthening
[]
[]
dewi
[]
[]
teach
[]
[]
minimally
[]
[]
invasive
[]
[]
instrument
[]
[]
veins
[]
[]
californialos
[]
[]
researcher
[]
[]
extremism
[]
[]
hatoyama
[]
[]
politically
[]
[]
heaviest
[]
[]
crippling
[]
[]
flooding
[]
[]
implants
[]
[]
micta
[]
[]
universities
[]
[]
ducks
[]
[]
routemaster
[]
[]
standing
[]
[]
balearic
[]
[]
superior
[]
[]
gately
[]
[]
pulmonary
[]
[]
edema
[]
[]
sees
[]
[]
abandoned
[]
[]
esfandiari
[]
[]
deflected
[]
[]
tie
[]
[]
ketsana
[]
[]
vodafone
[]
[]
payasyougo
[]
[]
bills
[]
[]
intakes
[]
[]
pulling
[]
[]
bedcovers
[]
[]
robot

Pour certaines phrases il est impossible de prédire le mot manquant car l'algorithme nous renvoie trop de mots possibles. 
On peut penser à regarder le tag du mot manquant à prédire à la place.

In [144]:
tag_predit =predire_tag(6, word_count, string, tag_count,'nn', 'nnp')

tag prédit du mot manquant :  nnps


## Amélioration du modèle : Modèle Tri-gramme

**Pour améliorer la  prédiction des mots on peut utiliser plus de contexte : tri-gramme**

Bien que le modèle bi-gramme fonctionne bien pour la détection de l'endroit et la prédiction du mot manquant, il peut donc etre interressant de capter plus de contexte en prenant un voisinage plus grand pour éviter d'obtenir le même schéma de mots et par conséquent afin d'affiner la prédiction des mots sur le jeu de tests. 
La condition est d'avoir un grand jeu d'entraînement.


**Transition Word Count:**

Voici la manière pour obtenir la probabilité de transition suivant le modèle tri-gramme:

Pour cette ordre précis : ... mot1 mot2 mot3  ....
#### P(mot3|mot2,mot1) = Count(mot1, mot2, mot3)/ Count(mot1,mot2) 


D'une façon similaire que celle effectuer sur le modèle bi-gramme le mot serait prédit de cette manière : 
- On trouve d'abord l'indice  du mot manquant via le bigram POS TAG par exemple,
- On détermine le mot: "mot1 mot2 **missing_word** mot3 mot4" en calculant argmax P(mot4| mot3, mot_test) * P(mot_test|mot2, mot1) pour chaque mot du vocabulaire du corpus (avec pour condition une probabilité >0)

ou encore argmax P(mot4| mot3, mot_test) * P(mot_test|mot2)  pour une suite de 4 mots ou encore armgax P(mot3| mot_test) * P(mot_test|mot2, mot1) 


In [94]:
def blanc(text):
    text=text.replace("  ", " ")
    return text

In [95]:
text_file = sc.textFile("D:/Projet-Spark/xaa_train_2000_ligne")
text_file= text_file.map(parseAndRemoveStopWords)
text_file= text_file.map(balise)
text_file= text_file.map(blanc)

trigrams = text_file.flatMap(lambda line: line.split(".")) \
                .map(lambda word : word.lower())  \
                .map(lambda line: line.strip().split(" ")) \
                .flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:], xs[2:])))


tri= trigrams.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

In [96]:
trigram = tri.collect()
tri.take(5)


[(('the', 'us', 'centers'), 1),
 (('us', 'centers', 'for'), 1),
 (('for', 'disease', 'control'), 2),
 (('disease', 'control', 'and'), 2),
 (('advised', 'school', 'systems'), 1)]

In [97]:
trigram

[(('the', 'us', 'centers'), 1),
 (('us', 'centers', 'for'), 1),
 (('for', 'disease', 'control'), 2),
 (('disease', 'control', 'and'), 2),
 (('advised', 'school', 'systems'), 1),
 (('school', 'systems', 'to'), 1),
 (('systems', 'to', 'close'), 1),
 (('close', 'if', 'outbreaks'), 1),
 (('if', 'outbreaks', 'occurred'), 1),
 (('outbreaks', 'occurred', 'then'), 1),
 (('occurred', 'then', 'reversed'), 1),
 (('reversed', 'itself', 'saying'), 1),
 (('saying', 'the', 'apparent'), 1),
 (('the', 'apparent', 'mildness'), 1),
 (('mildness', 'of', 'the'), 1),
 (('the', 'virus', 'meant'), 1),
 (('virus', 'meant', 'most'), 1),
 (('and', 'day', 'care'), 1),
 (('stay', 'open', 'even'), 1),
 (('open', 'even', 'if'), 1),
 (('if', 'they', 'had'), 1),
 (('had', 'confirmed', 'cases'), 1),
 (('cases', 'of', 'swine'), 1),
 (('when', 'ms', 'winfrey'), 1),
 (('somers', 'to', 'share'), 1),
 (('to', 'share', 'her'), 1),
 (('controversial', 'views', 'about'), 1),
 (('views', 'about', 'bioidentical'), 1),
 (('hormon

In [116]:
from pyspark.sql.types import *
sqlContext = SQLContext(sc)
from pyspark.sql import SQLContext
#df = SQLContext.createDataFrame(bigr, ["label", "valeur"])
trig = sqlContext.createDataFrame(trigram, ["Triplet", "valeur_t"])
trig.show(5)

+--------------------+--------+
|             Triplet|valeur_t|
+--------------------+--------+
|  [the, us, centers]|       1|
|  [us, centers, for]|       1|
|[for, disease, co...|       2|
|[disease, control...|       2|
|[advised, school,...|       1|
+--------------------+--------+
only showing top 5 rows



In [118]:
from pyspark.sql.functions import lit

from pyspark.sql.functions import udf
mot1f = udf(mot1, StringType())
mot2f = udf(mot2, StringType())
mot3f = udf(mot3, StringType())

trig= trig.select("Triplet", "valeur_t")
trig1= trig.withColumn('mot1',mot1f(trig['Triplet']))
trig1= trig1.withColumn('mot2',mot2f(trig['Triplet']))
trig1= trig1.withColumn('mot3',mot3f(trig['Triplet']))

trig1 = trig1.withColumn("P(mot3|mot2,mot1)", lit(0))
trig1.show(10)

+--------------------+--------+---------+---------+---------+-----------------+
|             Triplet|valeur_t|     mot1|     mot2|     mot3|P(mot3|mot2,mot1)|
+--------------------+--------+---------+---------+---------+-----------------+
|  [the, us, centers]|       1|      the|       us|  centers|                0|
|  [us, centers, for]|       1|       us|  centers|      for|                0|
|[for, disease, co...|       2|      for|  disease|  control|                0|
|[disease, control...|       2|  disease|  control|      and|                0|
|[advised, school,...|       1|  advised|   school|  systems|                0|
|[school, systems,...|       1|   school|  systems|       to|                0|
|[systems, to, close]|       1|  systems|       to|    close|                0|
|[close, if, outbr...|       1|    close|       if|outbreaks|                0|
|[if, outbreaks, o...|       1|       if|outbreaks| occurred|                0|
|[outbreaks, occur...|       1|outbreaks

In [119]:
trig1.where(trig1.mot1=="debut").show(5)

+--------------------+--------+-----+------+-------+-----------------+
|             Triplet|valeur_t| mot1|  mot2|   mot3|P(mot3|mot2,mot1)|
+--------------------+--------+-----+------+-------+-----------------+
|[debut, fish, ran...|       1|debut|  fish| ranked|                0|
|[debut, amman, re...|       1|debut| amman|reuters|                0|
|[debut, brent, no...|       1|debut| brent|  north|                0|
|[debut, that, seems]|       1|debut|  that|  seems|                0|
|[debut, gordon, w...|       1|debut|gordon|   will|                0|
+--------------------+--------+-----+------+-------+-----------------+
only showing top 5 rows



In [120]:
pair_word_count_df =df1.drop("P(mot2|mot1)").drop("Pair")
pair_word_count_df.show(5)


+------+-------+-------+
|valeur|   mot1|   mot2|
+------+-------+-------+
|   287|  debut|    the|
|     1|     us|centers|
|     2|    for|disease|
|     2|disease|control|
|     2|control|    and|
+------+-------+-------+
only showing top 5 rows



#### jointure pair_word_count et triplet word_count  : si triplet.mot1 =pair.mot1 et triplet.mot2= pair.mot2 

In [121]:
trig2= trig1.join(pair_word_count_df, (trig1.mot1 ==pair_word_count_df.mot1) & (trig1.mot2 ==pair_word_count_df.mot2))


In [122]:
trig2.show(10)

+--------------------+--------+-------+-----------+--------------+-----------------+------+-------+-----------+
|             Triplet|valeur_t|   mot1|       mot2|          mot3|P(mot3|mot2,mot1)|valeur|   mot1|       mot2|
+--------------------+--------+-------+-----------+--------------+-----------------+------+-------+-----------+
|[a, remixed, vers...|       1|      a|    remixed|       version|                0|     1|      a|    remixed|
|[a, treasury, app...|       1|      a|   treasury|   appointment|                0|     1|      a|   treasury|
|[ableton, live, a...|       1|ableton|       live|         apple|                0|     1|ableton|       live|
|[actuary, andrew,...|       1|actuary|     andrew|         young|                0|     1|actuary|     andrew|
|[allstar, interac...|       1|allstar|interactive|      luncheon|                0|     1|allstar|interactive|
|[an, extremely, e...|       1|     an|  extremely|     effective|                0|     1|     an|  ext

### matrice de transition tri-gramme

In [125]:
trig3 =trig2.withColumn("P(mot3|mot2,mot1)", (F.col("valeur_t") / F.col("valeur")))
trig3= trig3.drop("valeur").drop("valeur_t").drop("Triplet")
trig3.show()

+---------+-----------+---------------+------------------+---------+-----------+
|     mot1|       mot2|           mot3| P(mot3|mot2,mot1)|     mot1|       mot2|
+---------+-----------+---------------+------------------+---------+-----------+
|        a|    remixed|        version|               1.0|        a|    remixed|
|        a|   treasury|    appointment|               1.0|        a|   treasury|
|  ableton|       live|          apple|               1.0|  ableton|       live|
|  actuary|     andrew|          young|               1.0|  actuary|     andrew|
|  allstar|interactive|       luncheon|               1.0|  allstar|interactive|
|       an|  extremely|      effective|               1.0|       an|  extremely|
|      and|     cheese|         makers|               0.5|      and|     cheese|
|      and|     cheese|         mashed|               0.5|      and|     cheese|
|      and|       drug|             or|0.3333333333333333|      and|       drug|
|      and|       drug| admi

#### La matrice de transition pour le modèle tri-gramme étant obtenu, on pourra donc réaliser des prédictions de la même manière que pour le modèle bi-gramme

## Conclusion 

##### Le modèle bi-gramme pos tag est donc adapté pour détecter les emplacements de mots manquants.
##### Le modèle bi-gramme mots et pos tag  fonctionne évidement sur le train.
##### En revanche, une grande taille du jeu d'entraînement est nécessaire pour que le modèle fonctionne. Il peut ne renvoyer aucun résultat lorsque le training set est petit un peu comme celui que l'on a choisi et notament si le vocabulaire de test n'est pas compris dans le train. 
##### De plus, le temps de prédiction est dépendant de la taille du vocabulaire d'entraînement. Il faut en effet parcourir tous le vocabulaire du corpus train, pour tester tous les probabilités, par exemple, pour un vocabulaire de taille 9000, cela prend 30 minutes environ.

##### A l'inverse si le modèle est entrainé sur un très gros corpus alors il peut être intéressant d'appliquer un modèle tri-gramme au vue de la multiplication des schémas de mots possible, de même pour la prédiction de l'indice manquant via les Pos Tag.

## Améliorations possibles

#### Voici les pistes qui peuvent être intéressantes à améliorer:

- **Amélioration de la partie regex**
- **Remplacer les identités nommées par un nom "génerique" prédictible de la même manière que les nombres**
- **Augmenter le nombre de lignes dans le train set. Cependant seulement augmentera grandement le temps de prédiction.**

